# Data Science Capstone Notebook

This notebook will mainly be used for the data science capstone project

In [3]:
!pip install BeautifulSoup4
!pip install requests
!pip install geopy
import bs4
import pandas as pd
import numpy as np
import requests
import numpy as np
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 112kB 6.2MB/s eta 0:00:01
Hello Capstone Project Course!


Reading the table from wikipedia into a dataframe using pandas:

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Iterating through the dataset and dropping all rows where the borough is not assigned:

In [5]:
for i in range(0,df.shape[0]):
    if df['Borough'][i]=='Not assigned':
        df.drop(i,axis=0,inplace=True)
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
print(df.shape)

(103, 3)


Reading the geospatial csv file into a dataframe:

In [7]:
loc_data = pd.read_csv('https://cocl.us/Geospatial_data')
loc_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Performing an inner merge on the first dataframe and the geospatial data:

In [8]:
df_merged=pd.merge(left=df, right=loc_data, left_on='Postal Code', right_on='Postal Code')
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Iterating through the dataset and dropping all rows that do not contain the substring 'Toronto' in the borough name:

In [9]:
for i in range(0,df_merged.shape[0]):
    if 'Toronto' not in df_merged['Borough'][i]:
        df_merged.drop(i,axis=0,inplace=True)
df_merged.reset_index(inplace=True, drop=True)
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [11]:
address = "Toronto, Ontario"
geolocator = Nominatim(user_agent="custom")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto is located at geographical coordinates: {}, {}.'.format(latitude, longitude))

Toronto is located at geographical coordinates: 43.6534817, -79.3839347.


In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
map_toronto


In [15]:
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [17]:
CLIENT_ID = 'WDPS1AL4CKYJKHFBWQ3SKDCUYT0PJBBXPHCSOGKFRBNJMX32' # your Foursquare ID
CLIENT_SECRET = 'D1G5OSUFZTBZFG3ELNKNVPOXZLZEXJSJG4TAVMCODPIPZDHR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WDPS1AL4CKYJKHFBWQ3SKDCUYT0PJBBXPHCSOGKFRBNJMX32
CLIENT_SECRET:D1G5OSUFZTBZFG3ELNKNVPOXZLZEXJSJG4TAVMCODPIPZDHR


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [20]:

toronto_venues = getNearbyVenues(names=df_merged['Neighborhood'], latitudes=df_merged['Latitude'],longitudes=df_merged['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1215, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,40,40,40,40,40,40
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",21,21,21,21,21,21
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",19,19,19,19,19,19
Central Bay Street,40,40,40,40,40,40
Christie,18,18,18,18,18,18
Church and Wellesley,40,40,40,40,40,40
"Commerce Court, Victoria Hotel",40,40,40,40,40,40
Davisville,40,40,40,40,40,40


In [26]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 213 unique categories.


In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1215, 213)


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.025,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.025641,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.052632,0.052632,0.052632,0.105263,0.157895,0.105263,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,Central Bay Street,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.025,0.0,0.0,0.0


In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.025,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.025641,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.052632,0.052632,0.052632,0.105263,0.157895,0.105263,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,Central Bay Street,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.025,0.0,0.0,0.0


In [34]:
print(toronto_grouped.shape)

(39, 213)


In [35]:
num_top_venues = 5
for n in toronto_grouped['Neighborhood']:
    print("----"+n+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == n].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0                 Café  0.08
1   Seafood Restaurant  0.05
2           Restaurant  0.05
3          Coffee Shop  0.05
4  Japanese Restaurant  0.05


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.08
1     Coffee Shop  0.08
2       Gift Shop  0.05
3  Sandwich Place  0.05
4      Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0     Burrito Place  0.10
1     Grocery Store  0.05
2  Sushi Restaurant  0.05
3        Comic Shop  0.05
4       Pizza Place  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.16
1     Boat or Ferry  0.11
2    Airport Lounge  0.11
3  Airport Terminal  0.11
4       Coffee Shop  0.11


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.1

In [36]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(39, 11)

In [38]:
kmeans= KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [41]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 2 7 5 2 9 7 0 7 7]
39


In [43]:

neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)
toronto_merged = df_merged
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Pub,Theater,Breakfast Spot,Mexican Restaurant,Distribution Center,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sandwich Place,Burrito Place,Park,Diner,Bookstore,Burger Joint,Italian Restaurant,Creperie,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Cosmetics Shop,Coffee Shop,Theater,Fast Food Restaurant,Café,Tea Room,Hotel,Bakery,Spa
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Restaurant,Farmers Market,Gastropub,Coffee Shop,Japanese Restaurant,Furniture / Home Store,Camera Store,Bakery,Creperie
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Bakery,Gastropub,Pub,Ramen Restaurant,Café,Health Food Store,Ice Cream Shop,Indian Restaurant,Mexican Restaurant,Cheese Shop


In [44]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)

map_clusters